# Set up the environment

In [1]:
%%sh
pip install -q pip --upgrade
pip install -q sagemaker --upgrade --user
pip install -q --upgrade opencv-python

ERROR: awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.13.45 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import os, time
import datetime
from sagemaker import get_execution_role

DEFAULT_ARN = 'arn:aws:iam::780410349667:role/service-role/AmazonSageMaker-ExecutionRole-20191202T133391'
ROLE = get_execution_role()
ROLE

'arn:aws:iam::780410349667:role/service-role/AmazonSageMaker-ExecutionRole-20191202T133391'

# Create the SageMaker Session

In [2]:
import sagemaker as sage
print(sage.__version__)
session = sage.Session()

1.49.0


# Get Docker Image Location from ECR

In [3]:
# Docker Image Name
image = 'sagemaker-tf-2-serving'

# Get the account ID
account = session.boto_session.client('sts').get_caller_identity()['Account']

# Get the region
region = session.boto_session.region_name

# Get the custom Image
ECR_IMAGE = f'{account}.dkr.ecr.{region}.amazonaws.com/{image}:latest'
ECR_IMAGE

'780410349667.dkr.ecr.eu-west-2.amazonaws.com/sagemaker-tf-2-serving:latest'

## 

# Generate Model.tar.gz and Upload Model to S3

In [4]:
"""import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)"""

"import tarfile\nwith tarfile.open('model.tar.gz', mode='w:gz') as archive:\n    archive.add('export', recursive=True)"

In [5]:
# MODEL_DATA = session.upload_data(path='model.tar.gz', key_prefix='model')
MODEL_DATA = 's3://' + session.default_bucket() + '/model/model.tar.gz'

In [6]:
MODEL_DATA

's3://sagemaker-eu-west-2-780410349667/model/model.tar.gz'

# Create TensorFlow Serving Model

In [7]:
from sagemaker.tensorflow.serving import Model

MODEL = Model(model_data=MODEL_DATA, 
              role=ROLE,
              image=ECR_IMAGE,
              framework_version='2.0.0',
              sagemaker_session=session)

# Deploy the Tensorflow Serving Model

In [8]:
import time

ENDPOINT_NAME = 'pornilarity-v1-endpoint'

PREDICTOR = MODEL.deploy(initial_instance_count=1, 
                         instance_type='ml.c5.xlarge',
                         endpoint_name=ENDPOINT_NAME)

print(f"Model Deployed at: {time.strftime('%Y-%M-%d:%H-%M-%S')}")

ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:eu-west-2:780410349667:endpoint/pornilarity-v1-endpoint".

# Predictions

## Get Local Test Image

In [10]:
import cv2
import numpy as np

In [75]:
# grayscale the image
image = cv2.imread("uploads/Nicole Aniston.png", cv2.IMREAD_GRAYSCALE)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (100, 100))
image = np.array(image).reshape(-1, 100, 100, 1)
image = image / 255.0

image.shape

(1, 100, 100, 1)

### Local Predictions

In [76]:
# PREDICTOR.content_type = 'application/json'

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
# print(PREDICTOR.predict(image))
# print(f"CloudWatch Log Time: {time.strftime('%Y-%M-%d:%H-%M-%S')}")

## Local Client Request using boto3

### Format payload into correct json format

In [77]:
import json, codecs, boto3, io

JSON_FILE = "payload.json"
CLIENT = boto3.client('runtime.sagemaker')

image_tensor = np.asarray(image).tolist()
payload = {"instances": image_tensor}

# json.dump(payload, codecs.open(JSON_FILE, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

### Load in Json File containing payload

In [78]:
# Endpoint Parameters

CUSTOM_ATTRIBUTES = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"  # An example of a trace ID.                                 
CONTENT_TYPE = "application/json"                           # The MIME type of the input data in the request body.
ACCEPT = "application/json"                                 # The desired MIME type of the inference in the response.

In [79]:
response = CLIENT.invoke_endpoint(CustomAttributes=CUSTOM_ATTRIBUTES,
                                  EndpointName=ENDPOINT_NAME,
                                  ContentType=CONTENT_TYPE,
                                  Accept=ACCEPT,
                                  Body=json.dumps(payload))

response_body = response['Body'].read().decode("utf-8") # decodes byte stream into string format
response_body = " ".join(response_body.split()) # gets rid of all the duplicate whitespaces and newline characters

## Regular Expression to format into array

In [80]:
import re

regex = re.compile("(?![e])[a-z\"\}\{:\s\[\]]*")

PREDICTIONS = regex.sub("", response_body).split(',')

In [81]:
PREDICTIONS = np.array(PREDICTIONS)
PREDICTIONS = PREDICTIONS.astype(np.float64)

## Get Top5 Predicitons

In [82]:
with open('labels.txt', 'r') as file:
    LABELS = file.read().splitlines()

In [83]:
np.argmax(PREDICTIONS)

66

In [84]:
def get_top5(predictions, labels):
    top_5 = []
    count = 0
    while count < 5:
        max_index = np.argmax(predictions)
        class_label = labels[max_index]
        top_5.append(class_label)
        predictions = np.delete(predictions, max_index)
        count+=1
    return top_5

In [85]:
get_top5(PREDICTIONS, LABELS)

['Nicole Aniston', 'Julia Ann', 'Luna Star', 'Sammie Rhodes', 'Aiden Starr']

# Clean Up

In [ ]:
ENDPOINT_NAME = 'pornilarity-v1-endpoint'
session.delete_endpoint(endpoint_name=ENDPOINT_NAME)